In [50]:
import sqlalchemy
import requests
import pandas as pd
import time
import json

from bs4 import BeautifulSoup
from sqlalchemy import create_engine

In [72]:
engine = create_engine("mysql+mysqlconnector://root@localhost/test", echo=True)

In [66]:
def get_data(category, save_data):
    try:
        url = f"https://scrapeme.live/shop/?orderby={category}"
        respon = requests.get(url=url)
        
        if respon.status_code == 200:
            print(f"Category pokemon {category} success, process get ada")
            time.sleep(6)
            soup = BeautifulSoup(respon.content)
            div_id = soup.find('div', attrs={'id':'primary'})
            table_data = div_id.find('ul', attrs={'class':'products columns-4'})
            time.sleep(6)

            for data in table_data:
                name = data.find_next('h2')
                price = data.find_next('span').text
                img = data.find_next('img')
            
                if not name is None and img is not None:
                    save_data['pokemon_name'].append(name.text)
                    save_data['price'].append(price)
                    save_data['img'].append(img['src'])

            print(f"Category pokemon {category} success get ada")
        return save_data
    
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

In [67]:
category_url = ['rating','price','price-desc']
category_pokemon = {
    'rating':[],
    'expensive_price':[],
    'cheap_price':[],
}

for category in category_url:
    pokemon = {
        'pokemon_name':[],
        'price':[],
        'img':[]
    }
    
    if category == 'rating':
        data = get_data(category, pokemon)
        category_pokemon['rating'].append(data)
    elif category == 'price':
        data = get_data(category, pokemon)
        category_pokemon['cheap_price'].append(data)
    else:
        data = get_data(category, pokemon)
        category_pokemon['expensive_price'].append(data)

Category pokemon rating success, process get ada
Category pokemon rating success get ada
Category pokemon price success, process get ada
Category pokemon price success get ada
Category pokemon price-desc success, process get ada
Category pokemon price-desc success get ada


In [69]:


# Convert to a single list of dictionaries
data = []
for category, items in category_pokemon.items():
    for item in items:
        combined_item = {key: item[key] for key in item}
        combined_item['category'] = category
        data.append(combined_item)

# Flatten the list of dictionaries
flattened_data = []
for entry in data:
    for i in range(len(entry['pokemon_name'])):
        flattened_data.append({
            'pokemon_name': entry['pokemon_name'][i],
            'price': entry['price'][i],
            'img': entry['img'][i],
            'category': entry['category']
        })

# Create DataFrame
all_pokemon = pd.DataFrame(flattened_data)

# Display the DataFrame
all_pokemon


,pokemon_name,price,img,category
0,Ivysaur,£87.00,https://scrapeme.live/wp-content/uploads/2018/...,rating
1,Ivysaur,£87.00,https://scrapeme.live/wp-content/uploads/2018/...,rating
2,Venusaur,£105.00,https://scrapeme.live/wp-content/uploads/2018/...,rating
3,Venusaur,£105.00,https://scrapeme.live/wp-content/uploads/2018/...,rating
4,Charmander,£48.00,https://scrapeme.live/wp-content/uploads/2018/...,rating
...,...,...,...,...
91,Machoke,£27.00,https://scrapeme.live/wp-content/uploads/2018/...,cheap_price
92,Rhydon,£27.00,https://scrapeme.live/wp-content/uploads/2018/...,cheap_price
93,Rhydon,£27.00,https://scrapeme.live/wp-content/uploads/2018/...,cheap_price
94,Corsola,£27.00,https://scrapeme.live/wp-content/uploads/2018/...,cheap_price


In [70]:
all_pokemon.to_csv("Daftar_Pokemon.csv")

In [71]:
all_pokemon.to_json("Daftar_Pokemon.json")

In [73]:
#save to table mysql
conn = engine.connect()
trans = conn.begin()

try:
    all_pokemon.to_sql("assignment_web_scraping", 
                conn, if_exists='replace', 
                index=False)
    
    trans.commit()
except Exception as e:
    trans.rollback()
    print(f"Error: {e}")

2024-07-07 17:42:21,978 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-07-07 17:42:21,980 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:21,985 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-07-07 17:42:21,986 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:21,988 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-07-07 17:42:21,989 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:21,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-07 17:42:22,009 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`assignment_web_scraping`
2024-07-07 17:42:22,010 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:22,046 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`assignment_web_scraping`
2024-07-07 17:42:22,047 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:22,083 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-07-07 17:42:22,084 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-07 17:42:22,10